# CONVERT TOPOLOGIC GRAPH INTO AN RDF/BOT GRAPH

### This script builds a 3D model of a two-storey building as a CellComplex with apertures (doors and windows) and contents (furniture), converts it into a topologic graph, and, lastly, converts the topologic graph into an RDF/BOT graph. This is an experimental feature and we welcome your feedback on it.

#### This script requires topologicpy version 0.7.8 or newer

## Import Libraries

In [3]:
# Make sure you have pip installed rdflib
import sys
sys.path.append("C:/Users/sarwj/OneDrive - Cardiff University/Documents/GitHub/topologicpy/src")
from topologicpy.Vertex import Vertex
from topologicpy.Edge import Edge
from topologicpy.Wire import Wire
from topologicpy.Face import Face
from topologicpy.Shell import Shell
from topologicpy.Cell import Cell
from topologicpy.CellComplex import CellComplex
from topologicpy.Cluster import Cluster
from topologicpy.Topology import Topology
from topologicpy.Aperture import Aperture
from topologicpy.Graph import Graph
from topologicpy.Dictionary import Dictionary
from topologicpy.Helper import Helper
from topologicpy.Plotly import Plotly
import uuid
import hashlib
import json

## Check the topologicpy version

In [4]:
print("This script requires topologicpy version 0.7.7 or newer.")
print("You are using topologicpy version:",Helper.Version())

This script requires topologicpy version 0.7.7 or newer.
You are using topologicpy version: The version that you are using (0.8.15) is the latest version available on PyPI.


In [5]:
def assignTypesAndLabels(cellComplex, dictionary, typeKey="type", labelKey="label", volumeKey="hasVolume", areaKey="hasArea"):
    from topologicpy.Topology import Topology
    decomp = CellComplex.Decompose(cellComplex)
    keys = decomp.keys()
    for key in keys:
        elements = decomp[key]
        n = max(len(str(len(elements))),4)
        for i, element in enumerate(elements):
            d = Topology.Dictionary(element)
            labelValue = Dictionary.ValueAtKey(d, labelKey)
            typeValue = Dictionary.ValueAtKey(d, typeKey)
            uuid = Topology.UUID(element, namespace="http://github.com/wassimj/topologicpy/resources")
            if not typeValue:
                typeValue = dictionary[key]
            if labelValue == None:
                labelValue = str(uuid)
            if "face" in key.lower() or "aperture" in key.lower():
                areaValue = Face.Area(element)
                d = Dictionary.ByKeysValues([typeKey, labelKey, areaKey], [typeValue, labelValue, areaValue])
            elif "cell" in key.lower():
                volumeValue = Cell.Volume(element)
                d = Topology.Dictionary(element)
                d = Dictionary.SetValueAtKey(d, typeKey, typeValue)
                d = Dictionary.SetValueAtKey(d, volumeKey, volumeValue)
                d = Dictionary.SetValueAtKey(d, labelKey, labelValue)
                contents = Topology.Contents(element)
                m = max(len(str(len(contents))),4)
                for content in contents:
                    d1 = Topology.Dictionary(content)
                    labelValue1 = Dictionary.ValueAtKey(d1, labelKey)
                    if labelValue1 == None:
                        labelValue1 = str(uuid)
                    d1 = Dictionary.SetValueAtKey(d1, labelKey, labelValue1)
                    d1 = Dictionary.SetValueAtKey(d1, typeKey, "Content")
                    content = Topology.SetDictionary(content, d1)
            element = Topology.SetDictionary(element, d)

## Build the House

In [6]:
selectors = []
namespace = "http://github.com/wassimj/topologicpy/resources"
lr_origin = Vertex.ByCoordinates(0,0,0)
living_room = Cell.Prism(origin=lr_origin, width=3, length=5, height=3, placement="lowerleft")
cen = Topology.Centroid(living_room)
uuid = Topology.UUID(living_room, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Living Room"])
cen = Topology.SetDictionary(cen, d)
selectors.append(cen)

kt_origin = Vertex.ByCoordinates(0,5,0)
kitchen = Cell.Prism(origin=kt_origin, width=3, length=4, height=3, placement="lowerleft")
cen = Topology.Centroid(kitchen)
uuid = Topology.UUID(kitchen, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Kitchen"])
cen = Topology.SetDictionary(cen, d)
selectors.append(cen)

cr_origin = Vertex.ByCoordinates(3,2,0)
corridor = Cell.Prism(origin=cr_origin, width=2, length=6, height=3, placement="lowerleft")
cen = Topology.Centroid(corridor)
uuid = Topology.UUID(corridor, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Corridor"])
cen = Topology.SetDictionary(cen, d)
selectors.append(cen)

br_origin = Vertex.ByCoordinates(5,0,0)
bedroom = Cell.Prism(origin=br_origin, width=4, length=4, height=3, placement="lowerleft")
cen = Topology.Centroid(bedroom)
uuid = Topology.UUID(bedroom, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Bedroom"])
cen = Topology.SetDictionary(cen, d)
selectors.append(cen)

bt_origin = Vertex.ByCoordinates(5,4,0)
bathroom = Cell.Prism(origin=bt_origin, width=3, length=2, height=3, placement="lowerleft")
cen = Topology.Centroid(bathroom)
uuid = Topology.UUID(bathroom, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Bathroom"])
cen = Topology.SetDictionary(cen, d)
selectors.append(cen)

ho_origin = Vertex.ByCoordinates(5,6,0)
home_office = Cell.Prism(origin=ho_origin, width=4, length=3, height=3, placement="lowerleft")
cen = Topology.Centroid(home_office)
uuid = Topology.UUID(home_office, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Home Office"])
cen = Topology.SetDictionary(cen, d)
selectors.append(cen)

sw_origin = Vertex.ByCoordinates(3.2,8,0)
stairwell = Cell.Prism(origin=sw_origin, width=1.6, length=3.2, height=6, wSides=2, placement="lowerleft")
cen = Topology.Centroid(stairwell)
uuid = Topology.UUID(stairwell, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Stairwell"])
cen = Topology.SetDictionary(cen, d)
selectors.append(cen)

gd_flr_rooms = [living_room, kitchen, corridor, bedroom, bathroom, home_office]

lr_origin = Vertex.ByCoordinates(0,0,3)
living_room = Cell.Prism(origin=lr_origin, width=3, length=5, height=3, placement="lowerleft")
cen = Topology.Centroid(living_room)
uuid = Topology.UUID(living_room, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Living Room"])
cen = Topology.SetDictionary(cen, d)
selectors.append(cen)

kt_origin = Vertex.ByCoordinates(0,5,3)
kitchen = Cell.Prism(origin=kt_origin, width=3, length=4, height=3, placement="lowerleft")
cen = Topology.Centroid(kitchen)
uuid = Topology.UUID(kitchen, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Kitchen"])
cen = Topology.SetDictionary(cen, d)
selectors.append(cen)

cr_origin = Vertex.ByCoordinates(3,2,3)
corridor = Cell.Prism(origin=cr_origin, width=2, length=6, height=3, placement="lowerleft")
cen = Topology.Centroid(corridor)
uuid = Topology.UUID(corridor, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Corridor"])
cen = Topology.SetDictionary(cen, d)
selectors.append(cen)

br_origin = Vertex.ByCoordinates(5,0,3)
bedroom = Cell.Prism(origin=br_origin, width=4, length=4, height=3, placement="lowerleft")
cen = Topology.Centroid(bedroom)
uuid = Topology.UUID(bedroom, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Bedroom"])
cen = Topology.SetDictionary(cen, d)
selectors.append(cen)

bt_origin = Vertex.ByCoordinates(5,4,3)
bathroom = Cell.Prism(origin=bt_origin, width=3, length=2, height=3, placement="lowerleft")
cen = Topology.Centroid(bathroom)
uuid = Topology.UUID(bathroom, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Bathroom"])
cen = Topology.SetDictionary(cen, d)
selectors.append(cen)

ho_origin = Vertex.ByCoordinates(5,6,3)
home_office = Cell.Prism(origin=ho_origin, width=4, length=3, height=3, placement="lowerleft")
cen = Topology.Centroid(home_office)
uuid = Topology.UUID(home_office, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Home Office"])
cen = Topology.SetDictionary(cen, d)
selectors.append(cen)


fst_flr_rooms = [living_room, kitchen, corridor, bedroom, bathroom, home_office]

house = CellComplex.ByCells(gd_flr_rooms+fst_flr_rooms+[stairwell])

# Build the doors
orig = Vertex.ByCoordinates(0.2,5,0.05)
d2 = Face.Rectangle(origin=orig, width=2.6, length=2.10, placement="lowerleft")
d2 = Topology.Rotate(d2, origin=orig, axis=[1,0,0], angle=90)

orig = Vertex.ByCoordinates(3,3.4,0.05)
d3 = Face.Rectangle(origin=orig, width=2.10, length=1, placement="lowerleft")
d3 = Topology.Rotate(d3, origin=orig, axis=[0,1,0], angle=-90)

orig = Vertex.ByCoordinates(3,5.2,0.05)
d4 = Face.Rectangle(origin=orig, width=2.10, length=1, placement="lowerleft")
d4 = Topology.Rotate(d4, origin=orig, axis=[0,1,0], angle=-90)

orig = Vertex.ByCoordinates(5,2.8,0.05)
d5 = Face.Rectangle(origin=orig, width=2.10, length=1, placement="lowerleft")
d5 = Topology.Rotate(d5, origin=orig, axis=[0,1,0], angle=-90)

orig = Vertex.ByCoordinates(5,4.2,0.05)
d6 = Face.Rectangle(origin=orig, width=2.10, length=1, placement="lowerleft")
d6 = Topology.Rotate(d6, origin=orig, axis=[0,1,0], angle=-90)

orig = Vertex.ByCoordinates(5,6.2,0.05)
d7 = Face.Rectangle(origin=orig, width=2.10, length=1, placement="lowerleft")
d7 = Topology.Rotate(d7, origin=orig, axis=[0,1,0], angle=-90)

orig = Vertex.ByCoordinates(3.3,2,0.05)
d8 = Face.Rectangle(origin=orig, width=1.4, length=2.10, placement="lowerleft")
d8 = Topology.Rotate(d8, origin=orig, axis=[1,0,0], angle=90)
d9 = Topology.Translate(d8, 0, 9.2, 0)
door_centroid = Topology.Centroid(d8)
gd_flr_drs = [d2,d3,d4,d5,d6,d7]
fst_flr_dr = [Topology.Translate(door,0,0,3) for door in gd_flr_drs]
drs = gd_flr_drs + fst_flr_dr + [d8,d9]

# Set the dictionaries for the doors
doors = []
for door in drs:
    uuid = Topology.UUID(door, namespace=namespace)
    d = Dictionary.ByKeysValues(["label", "name", "type"], [str(uuid), "Door", "Door"])
    door = Topology.SetDictionary(door, d)
    doors.append(door)
door_cluster = Cluster.ByTopologies(doors)

# Build the windows
decomp = CellComplex.Decompose(house)
outer_walls = decomp["externalVerticalFaces"]
windows = []
for outer_wall in outer_walls:
    window = Topology.Scale(outer_wall, origin=Topology.Centroid(outer_wall), x=0.3, y=0.3, z=0.3)
    window = Topology.Translate(window, x=0, y=0, z=-0.4)
    uuid = Topology.UUID(door, namespace=namespace)
    d = Dictionary.ByKeysValues(["label", "name", "type"], [str(uuid), "Window", "Window"])
    window = Topology.SetDictionary(window, d)
    windows.append(window)
windows = [w for w in windows if Vertex.Distance(Topology.Centroid(w),door_centroid) > 0.2]
windows = [w for w in windows if Face.Area(w) > 0.5]

window_cluster = Cluster.ByTopologies(windows)
# Add the doors and windows to the house
house = Topology.AddApertures(house, doors+windows, exclusive=True, subTopologyType="face", tolerance=0.001)

# Add Furniture
bed = Cell.Prism(origin=Vertex.ByCoordinates(7.8,2,0), width=2, length=1.4, height=0.6, placement="bottom")
uuid = Topology.UUID(bed, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Bed"])
bed = Topology.SetDictionary(bed, d)

st1 = Cell.Prism(origin=Vertex.ByCoordinates(8.55,0.9,0), width=0.5, length=0.5, height=0.6, placement="bottom")
uuid = Topology.UUID(st1, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Side Table"])
st1 = Topology.SetDictionary(st1, d)

st2 = Cell.Prism(origin=Vertex.ByCoordinates(8.55,3.1,0), width=0.5, length=0.5, height=0.6, placement="bottom")
uuid = Topology.UUID(st2, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Side Table"])
st2 = Topology.SetDictionary(st2, d)

desk1 = Cell.Prism(origin=Vertex.ByCoordinates(8.4,7,0), width=0.7, length=1.5, height=0.6, placement="bottom")
uuid = Topology.UUID(desk1, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Desk"])
desk1 = Topology.SetDictionary(desk1, d)

ch1 = Cell.Prism(origin=Vertex.ByCoordinates(7.7,7,0), width=0.5, length=0.5, height=0.6, placement="bottom")
uuid = Topology.UUID(ch1, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Chair"])
ch1 = Topology.SetDictionary(ch1, d)

desk2 = Cell.Prism(origin=Vertex.ByCoordinates(6,8.4,0), width=1.5, length=0.7, height=0.6, placement="bottom")
uuid = Topology.UUID(desk2, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Desk"])
desk2 = Topology.SetDictionary(desk2, d)

ch2 = Cell.Prism(origin=Vertex.ByCoordinates(5.95,7.7,0), width=0.5, length=0.5, height=0.6, placement="bottom")
uuid = Topology.UUID(ch2, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Chair"])
ch2 = Topology.SetDictionary(ch2, d)

toilet = Cell.Prism(origin=Vertex.ByCoordinates(6.5,5.5,0), width=0.5, length=0.65, height=0.6, placement="bottom")
uuid = Topology.UUID(toilet, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Toilet"])
toilet = Topology.SetDictionary(toilet, d)

sink = Cell.Prism(origin=Vertex.ByCoordinates(5.6,5.6,0), width=0.6, length=0.5, height=0.6, placement="bottom")
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Sink"])
sink = Topology.SetDictionary(sink, d)

tub = Cell.Prism(origin=Vertex.ByCoordinates(7.45,5,0), width=0.8, length=1.7, height=0.6, placement="bottom")
uuid = Topology.UUID(tub, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Tub"])
tub = Topology.SetDictionary(tub, d)

sofa1 = Cell.Prism(origin=Vertex.ByCoordinates(1.5,2.8,0), width=0.8, length=0.8, height=0.6, placement="bottom")
uuid = Topology.UUID(sofa1, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Sofa"])
sofa1 = Topology.SetDictionary(sofa1, d)

sofa2 = Cell.Prism(origin=Vertex.ByCoordinates(0.6,1.25,0), width=0.8, length=2, height=0.6, placement="bottom")
uuid = Topology.UUID(sofa2, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Sofa"])
sofa2 = Topology.SetDictionary(sofa2, d)

st3 = Cell.Prism(origin=Vertex.ByCoordinates(0.7,2.65,0), width=0.5, length=0.5, height=0.6, placement="bottom")
uuid = Topology.UUID(st3, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Side Table"])
st3 = Topology.SetDictionary(st3, d)

tv = Cell.Prism(origin=Vertex.ByCoordinates(2.6,1.25,0), width=0.4, length=1, height=0.6, placement="bottom")
uuid = Topology.UUID(tv, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "TV"])
tv = Topology.SetDictionary(tv, d)

kt = Cell.Prism(origin=Vertex.ByCoordinates(1.5,6.5,0), width=1, length=1.4, height=0.6, placement="bottom")
uuid = Topology.UUID(kt, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Kitchen Island"])
kt = Topology.SetDictionary(kt, d)

cb = Cell.Prism(origin=Vertex.ByCoordinates(1.5,8.6,0), width=2.8, length=0.6, height=0.6, placement="bottom")
uuid = Topology.UUID(cb, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Kitchen Cabinets"])
cb = Topology.SetDictionary(cb, d)

furniture_gd = [bed, st1, st2, desk1, ch1, desk2, ch2, toilet, sink, tub, sofa1, sofa2, st3, tv, kt, cb]
furniture_1 = [Topology.Translate(f, 0, 0, 3) for f in furniture_gd]
for i, f in enumerate(furniture_1):
    d = Topology.Dictionary(furniture_gd[i])
    uuid = Topology.UUID(f, namespace=namespace)
    d = Dictionary.SetValueAtKey(d, "label", str(uuid))
    f = Topology.SetDictionary(f, d)
furniture = furniture_gd + furniture_1
house = Topology.AddContent(house, furniture, subTopologyType="cell", tolerance=0.001)
house = Topology.TransferDictionariesBySelectors(house, selectors, tranCells=True)

d = {
    "cells": "Space",
    "externalVerticalFaces": "External_Vertical_Wall",
    "internalVerticalFaces": "Internal_Vertical_Wall",
    "topHorizontalFaces": "Roof_Slab",
    "bottomHorizontalFaces": "Ground_Slab",
    "internalHorizontalFaces": "Internal_Floor_Slab",
    "externalInclinedFaces": "External_Inclined_Wall",
    "internalInclinedFaces": "Internal_Inclined_Wall",
    "externalVerticalApertures": "External_Vertical_Window",
    "internalVerticalApertures": "Internal_Vertical_Door",
    "topHorizontalApertures": "Skylight_Roof_Window",
    "bottomHorizontalApertures": "Floor_Window",
    "internalHorizontalApertures": "Floor_Window",
    "externalInclinedApertures": "External_Inclined_Window",
    "internalInclinedApertures": "Internal_Inclined_Window",
}
#Assign the appropriate element types
_ = assignTypesAndLabels(house, d, typeKey="type", labelKey="label", areaKey="hasArea", volumeKey="hasVolume")
print("Done building the house.")

Done building the house.


## Visualize the Graph of the House

In [7]:
house_graph = Graph.ByTopology(house,
                     direct=False,
                     directApertures=True,
                     viaSharedTopologies=True,
                     viaSharedApertures=True,
                     toExteriorTopologies=True,
                     toExteriorApertures=True,
                     toContents = True,
                     storeBREP = False)
data01 = Plotly.DataByTopology(house, faceOpacity=0.1)
data02 = Plotly.DataByTopology(door_cluster, faceColor="brown", faceOpacity=0.2)
data03 = Plotly.DataByTopology(window_cluster, faceColor="lightblue", faceOpacity=0.8)
data04 = Plotly.DataByTopology(Cluster.ByTopologies(furniture), faceColor="yellow", faceOpacity=0.1)
data05= Plotly.DataByTopology(Graph.Topology(house_graph), vertexSize=5, vertexColor="red", edgeWidth=2)
figure = Plotly.FigureByData(data01+data02+data03+data04+data05)
Plotly.Show(figure)

## Check the Dictionaries of the Cells

In [ ]:
house_cells = Topology.Cells(house)
for house_cell in house_cells:
    d = Topology.Dictionary(house_cell)
    print(Dictionary.Keys(d), Dictionary.Values(d))

['hasVolume', 'label', 'name', 'type'] [45.0, '22a7c2a2-3244-553d-941c-589402014528', 'Living Room', 'Space']
['hasVolume', 'label', 'name', 'type'] [36.0, '21195572-03a6-5c18-bb82-5899662be86a', 'Corridor', 'Space']
['hasVolume', 'label', 'name', 'type'] [36.0, 'c944be19-3c5b-54ad-be12-ef082fb453c8', 'Kitchen', 'Space']
['hasVolume', 'label', 'name', 'type'] [45.0, '8f3397f7-927a-58c9-9d63-b9a929432bf6', 'Living Room', 'Space']
['hasVolume', 'label', 'name', 'type'] [48.0, 'a6a9f788-ecfd-509e-ba5a-b93913a7cfb1', 'Bedroom', 'Space']
['hasVolume', 'label', 'name', 'type'] [18.0, '26dd4e99-d019-5d79-b1d0-5d201fbe1787', 'Bathroom', 'Space']
['hasVolume', 'label', 'name', 'type'] [36.0, 'a93c8b15-e595-5b74-890c-5ec79140bc99', 'Home Office', 'Space']
['hasVolume', 'label', 'name', 'type'] [30.72, 'ab9bd77f-5517-546e-ba63-0ba9caa3c44f', 'Stairwell', 'Space']
['hasVolume', 'label', 'name', 'type'] [36.0, '6b8e00a1-764e-57d8-8cf7-19bf67a37f09', 'Corridor', 'Space']
['hasVolume', 'label', 'name

## Check the JSON String of the Graph

In [ ]:
json_string = Graph.JSONString(graph=house_graph,
                   verticesKey="vertices",
                   edgesKey="edges",
                   vertexLabelKey="label",
                   edgeLabelKey="label",
                   xKey = "hasX",
                   yKey = "hasY",
                   zKey = "hasZ",
                   indent=4,
                   sortKeys=False,
                   mantissa=6)
print(json_string)

{
    "vertices": {
        "9d00b82e-dcff-5403-be73-20ee90c22f16": {
            "hasArea": 12.0,
            "hasX": 1.8,
            "hasY": 7.2,
            "hasZ": 0.0,
            "label": "9d00b82e-dcff-5403-be73-20ee90c22f16",
            "type": "Ground_Slab"
        },
        "648a62b3-b24d-5c3f-9642-f6bf23be07eb": {
            "hasArea": 3.0,
            "hasX": 3.0,
            "hasY": 8.5,
            "hasZ": 1.5,
            "label": "648a62b3-b24d-5c3f-9642-f6bf23be07eb",
            "type": "External_Vertical_Wall"
        },
        "a2f29a34-acf5-5485-bbc3-de252130421d": {
            "hasArea": 9.0,
            "hasX": 1.5,
            "hasY": 9.0,
            "hasZ": 1.5,
            "label": "a2f29a34-acf5-5485-bbc3-de252130421d",
            "type": "External_Vertical_Wall"
        },
        "cbae882a-6b8b-590c-a838-4eab306a8a46": {
            "hasArea": 0.81,
            "hasX": 1.51,
            "hasY": 9.01,
            "hasZ": 1.11,
            "label": "c

## Check the RDF/BOT String

In [ ]:
site_dictionary = Dictionary.ByKeysValues(["lon", "lat"], [51.48657, -31.8265])
bot_string = Graph.BOTString(graph = house_graph,
                             format = "turtle",
                             bidirectional = False,
                             includeAttributes = True,
                             includeLabel = True,
                             includeGeometry = False,
                             siteLabel = "Site_0001",
                             siteDictionary = site_dictionary,
                             buildingLabel = "Building_0001",
                             buildingDictionary = None , 
                             storeyPrefix = "Storey",
                             floorLevels = [],
                             vertexLabelKey = "label",
                             typeKey = "type",
                             verticesKey = "vertices",
                             edgesKey = "edges",
                             edgeLabelKey = "",
                             sourceKey = "source",
                             targetKey = "target",
                             xKey = "hasX",
                             yKey = "hasY",
                             zKey = "hasZ",
                             geometryKey = "brep",
                             spaceType = "space",
                             wallType = "wall",
                             slabType = "slab",
                             doorType = "door",
                             windowType = "window",
                             contentType = "content",
                             namespace = "http://github.com/wassimj/topologicpy/resources",
                             mantissa = 6
                             )
print(bot_string)

@prefix bot: <https://w3id.org/bot#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix top: <http://github.com/wassimj/topologicpy/resources> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<Site_0001> a bot:Site ;
    rdfs:label "Site_0001" ;
    top:lat "-31.8265"^^xsd:float ;
    top:lon "51.48657"^^xsd:float ;
    bot:hasBuilding <Building_0001> .

top:0935db08-f4e9-5267-a80e-2b8b92cd47e2 a bot:Slab ;
    rdfs:label "0935db08-f4e9-5267-a80e-2b8b92cd47e2" ;
    top:hasArea "15.0"^^xsd:float ;
    top:hasX "1.8"^^xsd:float ;
    top:hasY "2.4"^^xsd:float ;
    top:hasZ "3.0"^^xsd:float ;
    bot:interfaceOf top:22a7c2a2-3244-553d-941c-589402014528,
        top:8f3397f7-927a-58c9-9d63-b9a929432bf6 .

top:09b63d33-5ecf-5c12-9b39-d179e0b3a4db a bot:Wall ;
    rdfs:label "09b63d33-5ecf-5c12-9b39-d179e0b3a4db" ;
    top:hasArea "12.0"^^xsd:float ;
    top:hasX "7.0"^^xsd:float ;
    top:hasY "0.0"^^xsd:float ;
    top:hasZ "1.5"^^xsd:float ;
    bot:connectsTo top:cb

## Export the RDF/BOT String to a file (Change the path!)

In [ ]:

path = "C:/Users/wassimj/Downloads/house.ttl" # Change Me!

status = Graph.ExportToBOT(graph = house_graph,
                            path = path,
                            format = "turtle",
                            overwrite = True,
                            bidirectional = False,
                            includeAttributes = True,
                            includeLabel = True,
                            includeGeometry = False,
                            siteLabel = "Site_0001",
                            siteDictionary = site_dictionary,
                            buildingLabel = "Building_0001",
                            buildingDictionary = None , 
                            storeyPrefix = "Storey",
                            floorLevels = [],
                            vertexLabelKey = "label",
                            typeKey = "type",
                            verticesKey = "vertices",
                            edgesKey = "edges",
                            edgeLabelKey = "",
                            sourceKey = "source",
                            targetKey = "target",
                            xKey = "hasX",
                            yKey = "hasY",
                            zKey = "hasZ",
                            geometryKey = "brep",
                            spaceType = "space",
                            wallType = "wall",
                            slabType = "slab",
                            doorType = "door",
                            windowType = "window",
                            contentType = "content",
                            namespace = "http://github.com/wassimj/topologicpy/resources",
                            mantissa = 6
                            )
if status:
    print("Success! Your RDF/BOT Graph has been save to:"+path)
else:
    print("Sorry! Something went wrong. Please try again.")

Success! Your RDF/BOT Graph has been save to:C:/Users/wassimj/Downloads/house.ttl
